In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler


In [ ]:
activity_data = pd.read_csv('activity_data.csv')
activity_data = activity_data.drop(['Date', 'Location'], axis=1)  # drop non-numerical columns
activity_data = activity_data.values  # convert to numpy array
activity_data = activity_data.astype('float32')  # convert to float32 for compatibility


In [ ]:
train_data = activity_data[:800, :-1]
train_labels = activity_data[:800, -1]
test_data = activity_data[800:, :-1]
test_labels = activity_data[800:, -1]


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)


In [ ]:
window_size = 10

X_train = []
y_train = []

for i in range(window_size, len(train_data_scaled)):
    X_train.append(train_data_scaled[i - window_size:i, :])
    y_train.append(train_labels[i])

X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_test = []
y_test = []

for i in range(window_size, len(test_data_scaled)):
    X_test.append(test_data_scaled[i - window_size:i, :])
    y_test.append(test_labels[i])

X_test, y_test = np.array(X_test), np.array(y_test)


In [ ]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
])


In [ ]:
model.compile(optimizer='adam', loss='mse')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
predicted_labels = model.predict(X_test)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
